<a href="https://colab.research.google.com/github/mahdi-alalawi/abo-israa/blob/master/Mahdimt5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive


def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')
            #  ^-- هذا السطر يستبدل الفواصل ويحول إلى أرقام

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()







    # بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # ... (بقية الكود الخاص بك) ...

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target'] # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # ... (بقية الكود الخاص بك) ...



    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف







  # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    # This is now done for both X_train and X_test
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna() # Drop NaNs from X_test as well
    y_test = y_test[X_test.index] # Align y_test with X_test after dropping NaNs




    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive


def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')
            #  ^-- هذا السطر يستبدل الفواصل ويحول إلى أرقام

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()







    # بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # ... (بقية الكود الخاص بك) ...

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 1  # إنشاء عمود 'target' بقيم 0

    target = data['target'] # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # ... (بقية الكود الخاص بك) ...



    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف







  # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    # This is now done for both X_train and X_test
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna() # Drop NaNs from X_test as well
    y_test = y_test[X_test.index] # Align y_test with X_test after dropping NaNs




    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive
from sklearn.metrics import mean_squared_error

# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""





    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN
    features = data.drop(['target'], axis=1)  # الميزات

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    target = data['target']  # الهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive
from sklearn.metrics import mean_squared_error

# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""





    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN
    features = data.drop(['target'], axis=1)  # الميزات

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    target = data['target']  # الهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive


def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')
            #  ^-- هذا السطر يستبدل الفواصل ويحول إلى أرقام

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()







    # بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # ... (بقية الكود الخاص بك) ...

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target'] # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # ... (بقية الكود الخاص بك) ...



    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف







  # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    # This is now done for both X_train and X_test
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna() # Drop NaNs from X_test as well
    y_test = y_test[X_test.index] # Align y_test with X_test after dropping NaNs




    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)




In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive


def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')
            #  ^-- هذا السطر يستبدل الفواصل ويحول إلى أرقام

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()







    # بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # ... (بقية الكود الخاص بك) ...

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target'] # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # ... (بقية الكود الخاص بك) ...



    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف







  # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    # This is now done for both X_train and X_test
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna() # Drop NaNs from X_test as well
    y_test = y_test[X_test.index] # Align y_test with X_test after dropping NaNs




    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive


def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')
            #  ^-- هذا السطر يستبدل الفواصل ويحول إلى أرقام

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()







    # بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # ... (بقية الكود الخاص بك) ...

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target'] # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # ... (بقية الكود الخاص بك) ...



    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف







  # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    # This is now done for both X_train and X_test
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna() # Drop NaNs from X_test as well
    y_test = y_test[X_test.index] # Align y_test with X_test after dropping NaNs




    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)

In [ ]:
# هذا الكود الاصغر مال مبتدأين
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive
from sklearn.metrics import mean_squared_error

# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""





    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN
    features = data.drop(['target'], axis=1)  # الميزات

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    target = data['target']  # الهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive


def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')
            #  ^-- هذا السطر يستبدل الفواصل ويحول إلى أرقام

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()







    # بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # ... (بقية الكود الخاص بك) ...

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target'] # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # ... (بقية الكود الخاص بك) ...



    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف







  # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    # This is now done for both X_train and X_test
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna() # Drop NaNs from X_test as well
    y_test = y_test[X_test.index] # Align y_test with X_test after dropping NaNs




    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)



In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive


def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')
            #  ^-- هذا السطر يستبدل الفواصل ويحول إلى أرقام

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()







    # بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # ... (بقية الكود الخاص بك) ...

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target'] # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # ... (بقية الكود الخاص بك) ...



    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف







  # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    # This is now done for both X_train and X_test
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna() # Drop NaNs from X_test as well
    y_test = y_test[X_test.index] # Align y_test with X_test after dropping NaNs




    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target']  # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna()  # Drop NaNs from X_test as well
    y_test = y_test[X_test.index]  # Align y_test with X_test after dropping NaNs

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive
from datetime import datetime

# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'

        # تحديد تنسيق التاريخ
        date_parser = lambda x: datetime.strptime(x, '%Y-%m-%d')  # تأكد من تعديل التنسيق حسب البيانات لديك

        data = pd.read_csv(file_path, index_col='Price', parse_dates=['Price'], date_parser=date_parser)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target']  # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna()  # Drop NaNs from X_test as well
    y_test = y_test[X_test.index]  # Align y_test with X_test after dropping NaNs

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive
from datetime import datetime

# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'

        # قراءة البيانات مع تجاهل عمود "Ticker"
        data = pd.read_csv(file_path, usecols=['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], parse_dates=['Price'])

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target']  # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna()  # Drop NaNs from X_test as well
    y_test = y_test[X_test.index]  # Align y_test with X_test after dropping NaNs

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# قراءة البيانات من الصورة
data = pd.DataFrame({
    'Price Ticker': ['GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F'],
    'Date': ['2000-08-30', '2000-08-31', '2000-09-01', '2000-09-05', '2000-09-06', '2000-09-07', '2000-09-08', '2000-09-11', '2000-09-12'],
    'Adj Close': [273.8999939, 278.2999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Close': [273.8999939, 278.2999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'High': [273.8999939, 278.2999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Low': [273.8999939, 274.7999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Open': [273.8999939, 274.7999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Volume': [0, 0, 0, 2, 0, 125, 0, 0, 0]
})

# تحويل التاريخ إلى نوع datetime
data['Date'] = pd.to_datetime(data['Date'])

# ترتيب البيانات حسب التاريخ
data = data.sort_values('Date')

# عرض البيانات
print(data)

# رسم بياني للبيانات
plt.figure(figsize=(12, 6))
plt.plot(data['Date'], data['Close'])
plt.title('Gold Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# قراءة البيانات من الصورة
data = pd.DataFrame({
    'Price Ticker': ['GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F'],
    'Date': ['2000-08-30', '2000-08-31', '2000-09-01', '2000-09-05', '2000-09-06', '2000-09-07', '2000-09-08', '2000-09-11', '2000-09-12'],
    'Adj Close': [273.8999939, 278.2999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Close': [273.8999939, 278.2999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'High': [273.8999939, 278.2999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Low': [273.8999939, 274.7999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Open': [273.8999939, 274.7999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Volume': [0, 0, 0, 2, 0, 125, 0, 0, 0]
})

# تحويل التاريخ إلى نوع datetime
data['Date'] = pd.to_datetime(data['Date'])

# ترتيب البيانات حسب التاريخ
data = data.sort_values('Date')

# عرض البيانات
print(data)

# رسم بياني للبيانات
plt.figure(figsize=(12, 6))
plt.plot(data['Date'], data['Close'])
plt.title('Gold Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# قراءة البيانات من الصورة
data = pd.DataFrame({
    'Price Ticker': ['GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F', 'GC=F'],
    'Date': ['2000-08-30', '2000-08-31', '2000-09-01', '2000-09-05', '2000-09-06', '2000-09-07', '2000-09-08', '2000-09-11', '2000-09-12'],
    'Adj Close': [273.8999939, 278.2999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Close': [273.8999939, 278.2999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'High': [273.8999939, 278.2999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Low': [273.8999939, 274.7999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Open': [273.8999939, 274.7999878, 277.0, 275.7999878, 274.2000122, 274.0, 273.2999878, 273.1000061, 272.8999939],
    'Volume': [0, 0, 0, 2, 0, 125, 0, 0, 0]
})

# تحويل التاريخ إلى نوع datetime
data['Date'] = pd.to_datetime(data['Date'])

# ترتيب البيانات حسب التاريخ
data = data.sort_values('Date')

# عرض البيانات
print(data)

# رسم بياني للبيانات
plt.figure(figsize=(12, 6))
plt.plot(data['Date'], data['Close'])
plt.title('Gold Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)
plt.show()

In [ ]:
# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target']  # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna()  # Drop NaNs from X_test as well
    y_test = y_test[X_test.index]  # Align y_test with X_test after dropping NaNs

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)

In [ ]:



# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive





# تحميل البيانات من Google Drive

# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, parse_dates=True)

        # طباعة أسماء الأعمدة للتحقق منها
        print("أسماء الأعمدة الموجودة في البيانات:")
        print(data.columns)

        # تجاهل الصفوف التي تحتوي على 'Ticker' أو 'Date' في عمود 'Price'
        data = data[~data['Price'].isin(['Ticker', 'Date'])]

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None










# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target']  # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna()  # Drop NaNs from X_test as well
    y_test = y_test[X_test.index]  # Align y_test with X_test after dropping NaNs

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive

# تحميل البيانات من Google Drive
def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, parse_dates=True)

        # طباعة أسماء الأعمدة للتحقق منها
        print("أسماء الأعمدة الموجودة في البيانات:")
        print(data.columns)

        # تجاهل الصفوف التي تحتوي على 'Ticker' أو 'Date' في عمود 'Price'
        data = data[~data['Price'].isin(['Ticker', 'Date'])]

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None










# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()






def convert_data_types(data):
    """تحويل الأعمدة إلى الأنواع المناسبة."""
    for col in data.select_dtypes(include=['object']).columns:
        try:
            data[col] = pd.to_numeric(data[col], errors='coerce')
        except ValueError:
            continue  # إذا لم يكن بالإمكان تحويل العمود، تجاهله
    return data




# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف

    # تحقق مما إذا كان العمود 'target' موجودًا، وإذا لم يكن موجودًا، فقم بإنشائه باستخدام الأصفار
    if 'target' not in data:
        data['target'] = 0  # إنشاء عمود 'target' بقيم 0

    target = data['target']  # الآن يجب أن يكون Pandas Series

    # تحويل الفهرس إلى أرقام صحيحة قبل إزالة الصفوف
    features = features.reset_index(drop=True)
    target = target.reset_index(drop=True)

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    data = convert_data_types(data)


    # إعداد الميزات والهدف
    features = data.drop(['target'], axis=1, errors='ignore')  # تأكد من وجود العمود المستهدف
    target = data['target'] if 'target' in data else np.zeros(len(data))  # في حالة عدم وجود العمود المستهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # إزالة الصفوف التي تحتوي على أي قيم NaN في الميزات أو الهدف
    X_train = X_train.dropna()
    y_train = y_train[X_train.index]  # الاحتفاظ بقيم الهدف المقابلة للميزات المتبقية
    X_test = X_test.dropna()  # Drop NaNs from X_test as well
    y_test = y_test[X_test.index]  # Align y_test with X_test after dropping NaNs

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # نموذج Random Forest
    model_rf = RandomForestRegressor()
    model_rf.fit(X_train, y_train)

    # نموذج SVM
    model_svr = SVR()
    model_svr.fit(X_train, y_train)

    # توليد الإشارات
    signals_xgb = model_xgb.predict(X_test)
    signals_rf = model_rf.predict(X_test)
    signals_svr = model_svr.predict(X_test)

    # دمج الإشارات في DataFrame
    signals_df = pd.DataFrame({
        'XGBoost': signals_xgb,
        'RandomForest': signals_rf,
        'SVM': signals_svr
    })

    # حساب متوسط الإشارات
    signals_df['Average_Signal'] = signals_df.mean(axis=1)

    # عرض بعض النتائج
    print("\nGenerated Trading Signals:")
    print(signals_df.head())

    # تقييم النماذج
    print("\nModel Evaluation:")
    print("XGBoost R^2 Score:", model_xgb.score(X_test, y_test))
    print("Random Forest R^2 Score:", model_rf.score(X_test, y_test))
    print("SVM R^2 Score:", model_svr.score(X_test, y_test))

    return signals_df

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive


def load_and_clean_data(file_id='YOUR_FILE_ID'):
    """تحميل البيانات من Google Drive وتنظيفها."""
    try:
        drive.mount('/content/drive')
        file_path = '/content/drive/My Drive/gold_usd_prices.csv'
        data = pd.read_csv(file_path, index_col='Price', parse_dates=True)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col].str.replace(',', ''), errors='coerce')
            #  ^-- هذا السطر يستبدل الفواصل ويحول إلى أرقام

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""





    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN
    features = data.drop(['target'], axis=1)  # الميزات

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    target = data['target']  # الهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown



import yfinance as yf

def load_and_clean_data(ticker='GC=F', start_date='2010-01-01', end_date='2023-01-01'):
    """تحميل البيانات من Yahoo Finance وتنظيفها."""
    try:
        # تحميل البيانات
        data = yf.download(ticker, start=start_date, end=end_date)

        # تحويل أعمدة محددة إلى أرقام
        for col in ['Adj Close', 'Open', 'Volume']:
            data[col] = pd.to_numeric(data[col], errors='coerce')

        print("أول 5 صفوف من إطار البيانات:")
        print(data.head())

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)
        print("\nإطار البيانات بعد التنظيف:")
        print(data.head())

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None




# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""





    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN
    features = data.drop(['target'], axis=1)  # الميزات

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    target = data['target']  # الهدف

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = load_and_clean_data()
    if data is not None:
        trading_signals = build_strategy(data)



In [ ]:
رد# prompt: انشئ كود يحمل البيانات من ياهو فايننس للذهب دولار لعام 2023 على فريم 5 دقائق و ينظف البيانات و يطبعها

# استيراد المكتبات اللازمة
import yfinance as yf
import pandas as pd

def load_and_clean_data(ticker='GC=F', start_date='2023-01-01', end_date='2023-12-31', interval="5m"):
    """تحميل البيانات من Yahoo Finance وتنظيفها."""
    try:
        # تحميل البيانات بفترة زمنية محددة و فاصل 5 دقائق
        data = yf.download(ticker, start=start_date, end=end_date, interval=interval)

        # تنظيف البيانات: إزالة القيم المفقودة
        data.dropna(inplace=True)

        return data
    except Exception as e:
        print(f"خطأ في تحميل أو تنظيف البيانات: {e}")
        return None

# تحميل البيانات
gold_data = load_and_clean_data()

# طباعة البيانات
if gold_data is not None:
gold_data

In [ ]:

from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive



import yfinance as yf
import pandas as pd

def get_and_clean_gold_prices(year):
    """
    Reads gold prices from Yahoo Finance for a given year,
    cleans the data, and prints it.
    """
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")

        # Cleaning the data:
        # 1. Remove rows with any missing values
        hist.dropna(inplace=True)
        # 2. Reset the index to default integer index
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
        # 3. (Optional) Convert 'Date' column to datetime objects
        #    hist['Date'] = pd.to_datetime(hist['Date'])

        print(hist)  # Print the cleaned data
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

#if __name__ == "__main__":
    #get_and_clean_gold_prices(2023)


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error  # تأكد من استيراد هذه الدالة

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""

    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()
    # ... (rest of the function code remains unchanged) ...


    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = get_and_clean_gold_prices(2023)
    if data is not None:
        trading_signals = build_strategy(data)




In [ ]:

from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive



import yfinance as yf
import pandas as pd

def get_and_clean_gold_prices(year):
    """
    Reads gold prices from Yahoo Finance for a given year,
    cleans the data, and prints it.
    """
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")

        # Cleaning the data:
        # 1. Remove rows with any missing values
        hist.dropna(inplace=True)
        # 2. Reset the index to default integer index
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
        # 3. (Optional) Convert 'Date' column to datetime objects
        #    hist['Date'] = pd.to_datetime(hist['Date'])

        print(hist)  # Print the cleaned data
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

#if __name__ == "__main__":
    #get_and_clean_gold_prices(2023)


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error  # تأكد من استيراد هذه الدالة

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""

    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()
    # ... (rest of the function code remains unchanged) ...


    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = get_and_clean_gold_prices(2023)
    if data is not None:
        trading_signals = build_strategy(data)




In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob yfinance

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive
import yfinance as yf

# تحميل البيانات من Yahoo Finance
def get_and_clean_gold_prices(year):
    """
    Reads gold prices from Yahoo Finance for a given year,
    cleans the data, and prints it.
    """
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")

        # Cleaning the data:
        hist.dropna(inplace=True)  # Remove rows with any missing values
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error  # تأكد من استيراد هذه الدالة

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""

    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

# مثال على كيفية استخدام الكود
if __name__ == "__main__":
    gold_data = get_and_clean_gold_prices(2023)
    if gold_data is not None:
        build_strategy(gold_data)


In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob yfinance

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from textblob import TextBlob
import gdown
from google.colab import drive
import yfinance as yf

# تحميل البيانات من Yahoo Finance
def get_and_clean_gold_prices(year):
    """
    Reads gold prices from Yahoo Finance for a given year,
    cleans the data, and prints it.
    """
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")

        # Cleaning the data:
        hist.dropna(inplace=True)  # Remove rows with any missing values
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error  # تأكد من استيراد هذه الدالة

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""

    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج Random Forest بدلاً من XGBoost
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

# مثال على كيفية استخدام الدوال
gold_data = get_and_clean_gold_prices(2023)
if gold_data is not None:
    build_strategy(gold_data)


In [ ]:

from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive



import yfinance as yf
import pandas as pd

def get_and_clean_gold_prices(year):
    """
    Reads gold prices from Yahoo Finance for a given year,
    cleans the data, and prints it.
    """
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")

        # Cleaning the data:
        # 1. Remove rows with any missing values
        hist.dropna(inplace=True)
        # 2. Reset the index to default integer index
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
        # 3. (Optional) Convert 'Date' column to datetime objects
        #    hist['Date'] = pd.to_datetime(hist['Date'])

        print(hist)  # Print the cleaned data
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

#if __name__ == "__main__":
    #get_and_clean_gold_prices(2023)


# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['Close'][i-1] and data['Close'][i-1] < data['Close'][i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'][i-1]))
        elif data['Close'][i] < data['Close'][i-1] and data['Close'][i-1] > data['Close'][i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'][i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'][i] > data['High'][i-1]:
            supply_zones.append((data.index[i], data['High'][i-1]))
        elif data['Close'][i] < data['Low'][i-1]:
            demand_zones.append((data.index[i], data['Low'][i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'][i] < data['Close'][i-1] and
            data['Close'][i+1] < data['Close'][i] and
            data['Close'][i+2] > data['Close'][i+1] and
            data['Close'][i+3] < data['Close'][i+2] and
            data['Close'][i+4] > data['Close'][i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error  # تأكد من استيراد هذه الدالة

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""

    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()
    # ... (rest of the function code remains unchanged) ...


    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = get_and_clean_gold_prices(2023)
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
import yfinance as yf

# تحميل البيانات من Yahoo Finance
def get_and_clean_gold_prices(year):
    """قراءة أسعار الذهب من Yahoo Finance لسنة معينة، وتنظيف البيانات."""
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")
        hist.dropna(inplace=True)
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]
    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue
    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(2, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = get_and_clean_gold_prices(2023)
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive

import yfinance as yf
import pandas as pd

def get_and_clean_gold_prices(year):
    """
    Reads gold prices from Yahoo Finance for a given year,
    cleans the data, and prints it.
    """
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")

        # Cleaning the data:
        # 1. Remove rows with any missing values
        hist.dropna(inplace=True)
        # 2. Reset the index to default integer index
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
        # 3. (Optional) Convert 'Date' column to datetime objects
        #    hist['Date'] = pd.to_datetime(hist['Date'])

        print(hist)  # Print the cleaned data
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

#if __name__ == "__main__":
    #get_and_clean_gold_prices(2023)

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error  # تأكد من استيراد هذه الدالة

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""

    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = get_and_clean_gold_prices(2023)
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
###تكملة الكود السابق
from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive

import yfinance as yf
import pandas as pd

def get_and_clean_gold_prices(year):
    """
    Reads gold prices from Yahoo Finance for a given year,
    cleans the data, and prints it.
    """
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")

        # Cleaning the data:
        # 1. Remove rows with any missing values
        hist.dropna(inplace=True)
        # 2. Reset the index to default integer index
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
        # 3. (Optional) Convert 'Date' column to datetime objects
        #    hist['Date'] = pd.to_datetime(hist['Date'])

        print(hist)  # Print the cleaned data
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

#if __name__ == "__main__":
    #get_and_clean_gold_prices(2023)

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error  # تأكد من استيراد هذه الدالة

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""

    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = get_and_clean_gold_prices(2023)
    if data is not None:
        trading_signals = build_strategy(data)


In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive

# تحميل البيانات من Google Drive

import yfinance as yf
import pandas as pd

def get_and_clean_gold_prices(year):
    """
    Reads gold prices from Yahoo Finance for a given year,
    cleans the data, and prints it.
    """
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")

        # Cleaning the data:
        # 1. Remove rows with any missing values
        hist.dropna(inplace=True)
        # 2. Reset the index to default integer index
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
        # 3. (Optional) Convert 'Date' column to datetime objects
        #    hist['Date'] = pd.to_datetime(hist['Date'])

        print(hist)  # Print the cleaned data
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

#if __name__ == "__main__":
    #get_and_clean_gold_prices(2023)

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error  # تأكد من استيراد هذه الدالة

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""

    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)

    # جمع البيانات من المفكرة الاقتصادية
    economic_calendar_data = fetch_economic_calendar()
    print("Economic Events:", economic_calendar_data)

    # رسم بياني لمناطق طلبات الشراء والبيع
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return predictions

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    # تحميل البيانات وتنفيذ الاستراتيجية
    data = get_and_clean_gold_prices(2023)
    if data is not None:
        trading_signals = build_strategy(data)



In [ ]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import yfinance as yf

# دالة لتحميل أسعار الذهب من Yahoo Finance
def get_and_clean_gold_prices(year):
    try:
        gold = yf.Ticker("GC=F")  # رمز العقود الآجلة للذهب
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")
        hist.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم مفقودة
        hist.index = pd.to_datetime(hist.index)  # تحويل الفهرس إلى كائنات datetime
        return hist
    except Exception as e:
        print(f"حدث خطأ: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]
    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity
        sentiments.append(score)
    return sentiments

# جمع البيانات من المفكرة الاقتصادية
def fetch_economic_calendar():
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue
    return events

# حساب مناطق Order Blocks
def calculate_order_blocks(data):
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)
    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error

def build_strategy(data):
    data['target'] = data['Close'].shift(-1)
    data.dropna(inplace=True)

    features = data.drop(['target'], axis=1)
    target = data['target']

    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    features.dropna(inplace=True)
    target = target[features.index]

    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    predictions = model_xgb.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='الأسعار الفعلية', color='blue')
    plt.plot(y_test.index, predictions, label='الأسعار المتنبأ بها', color='orange')
    plt.title('الأسعار الفعلية مقابل الأسعار المتنبأ بها')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.show()

    bullish_blocks, bearish_blocks = calculate_order_blocks(data)
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    supply_zones, demand_zones = calculate_supply_demand(data)
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    data = calculate_rsi(data)
    data = calculate_moving_average(data)
    data = calculate_momentum(data)
    data = analyze_volume(data)
    wave_points = calculate_wolf_wave(data)
    data = create_features(data)
    data = calculate_price_channel(data)
    data = advanced_risk_management(data)

    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    return data, bullish_blocks, bearish_blocks, supply_zones, demand_zones, wave_points

if __name__ == "__main__":
    gold_data = get_and_clean_gold_prices(2023)
    news_titles = fetch_news()
    sentiments = analyze_sentiment(news_titles)
    economic_events = fetch_economic_calendar()
    results, bullish_blocks, bearish_blocks, supply_zones, demand_zones, wave_points = build_strategy(gold_data)

    print("نتائج تحليل المشاعر:")
    for title, sentiment in zip(news_titles, sentiments):
        print(f"{title}: {sentiment}")

    print("\nالأحداث الاقتصادية:")
    for event in economic_events:
        print(event)

    print("\nمناطق طلبات الشراء والبيع:")
    print("مناطق الطلب:", bullish_blocks)
    print("مناطق العرض:", bearish_blocks)
    print("مناطق العرض:", supply_zones)
    print("مناطق الطلب:", demand_zones)
    print("نقاط موجة الذئب:", wave_points)


In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import gdown
from google.colab import drive
import yfinance as yf

def get_and_clean_gold_prices(year):
    """
    Reads gold prices from Yahoo Finance for a given year,
    cleans the data, and prints it.
    """
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")

        # Cleaning the data:
        hist.dropna(inplace=True)  # Remove rows with any missing values
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# جمع الأخبار من Investing.com
def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    news_headers = soup.select('div.news-item-header')
    news_titles = [header.text.strip() for header in news_headers]

    return news_titles

# تحليل المشاعر من الأخبار باستخدام TextBlob
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    for title in news_titles:
        analysis = TextBlob(title)
        score = analysis.sentiment.polarity  # تحليل المشاعر
        sentiments.append(score)
    return sentiments

# تحليل المفكرة الاقتصادية
def fetch_economic_calendar():
    """جمع البيانات من المفكرة الاقتصادية"""
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    events = []
    for event in soup.select('table.ecTable tr'):
        try:
            date = event.select_one('td.date').text.strip()
            impact = event.select_one('td.impact').text.strip()
            title = event.select_one('td.event').text.strip()
            events.append((date, impact, title))
        except AttributeError:
            continue

    return events

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI) باستخدام Pandas TA
def calculate_rsi(data, period=14):
    """حساب RSI"""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك باستخدام Pandas TA
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum) باستخدام Pandas TA
def calculate_momentum(data, period=10):
    """حساب الزخم"""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
from sklearn.metrics import mean_squared_error  # تأكد من استيراد هذه الدالة

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""

    # إعداد الميزات والهدف
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم
    data = advanced_risk_management(data)

    # رسم مناطق الطلب والعرض
    احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    # إرجاع البيانات مع الميزات الجديدة
    return data, wave_points

# تنفيذ الدالة
if __name__ == "__main__":
    # تحميل البيانات
    gold_data = get_and_clean_gold_prices(2024)

    if gold_data is not None:
        # بناء الاستراتيجية
        final_data, wave_points = build_strategy(gold_data)


In [ ]:
from IPython import get_ipython
from IPython.display import display

# تثبيت المكتبات المطلوبة
!pip install pandas_ta gdown keras xgboost seaborn statsmodels textblob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import requests
from bs4 import BeautifulSoup
from sklearn.metrics import mean_squared_error
import yfinance as yf

# تحميل البيانات
def get_and_clean_gold_prices(year):
    """قراءة أسعار الذهب من Yahoo Finance وتنظيف البيانات."""
    try:
        gold = yf.Ticker("GC=F")  # رمز العقود الآجلة للذهب
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")
        hist.dropna(inplace=True)
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        return hist
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# جمع الأخبار
def fetch_news():
    """جمع الأخبار من موقع Investing.com."""
    url = "https://www.investing.com/news/commodities-news/gold"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    news_headers = soup.select('div.news-item-header')
    return [header.text.strip() for header in news_headers]

# تحليل المشاعر
def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob."""
    from textblob import TextBlob
    return [TextBlob(title).sentiment.polarity for title in news_titles]

# حساب Order Blocks
def calculate_order_blocks(data):
    """حساب مناطق Order Blocks."""
    bullish_blocks = []
    bearish_blocks = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
            bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
            bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    return bullish_blocks, bearish_blocks

# حساب Premium وDiscount
def calculate_premium_discount(data):
    """حساب Premium وDiscount."""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

# حساب مناطق العرض والطلب
def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب."""
    supply_zones = []
    demand_zones = []
    for i in range(1, len(data)):
        if data['Close'].iloc[i] > data['High'].iloc[i-1]:
            supply_zones.append((data.index[i], data['High'].iloc[i-1]))
        elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
            demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    return supply_zones, demand_zones

# حساب خطوط الدعم والمقاومة
def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة."""
    support = data['Low'].rolling(window=20).min()
    resistance = data['High'].rolling(window=20).max()
    return support, resistance

# حساب مستويات فيبوناتشي
def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي."""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

# حساب مؤشر القوة النسبية (RSI)
def calculate_rsi(data, period=14):
    """حساب RSI."""
    data['RSI'] = ta.rsi(data['Close'], length=period)
    return data

# حساب المتوسط المتحرك
def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك."""
    data['MA'] = ta.sma(data['Close'], length=period)
    return data

# حساب الزخم (Momentum)
def calculate_momentum(data, period=10):
    """حساب الزخم."""
    data['Momentum'] = ta.mom(data['Close'], length=period)
    return data

# تحليل حجم التداول
def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان."""
    data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
    return data

# حساب موجة الذئب
def calculate_wolf_wave(data):
    """حساب موجة الذئب."""
    wave_points = []
    for i in range(1, len(data)-4):
        if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
            data['Close'].iloc[i+1] < data['Close'].iloc[i] and
            data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
            data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
            data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
            wave_points.append(data.index[i])
    return wave_points

# إنشاء ميزات إضافية
def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر."""
    data['Price_Change'] = data['Close'].pct_change()
    data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
    data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
    return data

# حساب مؤشر القنوات السعرية
def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية."""
    data['Channel_Upper'] = data['High'].rolling(window=window).max()
    data['Channel_Lower'] = data['Low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data

# تحسين إدارة المخاطر
def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم."""
    data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
    data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
    data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
    return data

# دالة لحساب ورسم مناطق طلبات الشراء والبيع
def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    for i in range(2, len(البيانات)):
        if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
        elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
            مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

    # رسم الرسم البياني
    plt.figure(figsize=(12, 6))
    plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

    for منطقة in مناطق_طلبات_الشراء:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

    for منطقة in مناطق_طلبات_البيع:
        index, price = منطقة
        plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

    plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
    plt.xlabel('التاريخ')
    plt.ylabel('السعر')
    plt.legend()
    plt.grid(True)
    plt.show()

# بناء الاستراتيجية
def build_strategy(data):
    """بناء الاستراتيجية الكاملة."""
    data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
    data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

    features = data.drop(['target'], axis=1)  # الميزات
    target = data['target']  # الهدف

    # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
    for col in ['Close', 'High', 'Low']:
        features[col] = pd.to_numeric(features[col], errors='coerce')

    # إزالة الصفوف الفارغة بعد التحويل
    features.dropna(inplace=True)
    target = target[features.index]  # الحفاظ على تطابق الفهارس

    # تقسيم البيانات
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # نموذج XGBoost
    model_xgb = XGBRegressor()
    model_xgb.fit(X_train, y_train)

    # التنبؤ بالأسعار
    predictions = model_xgb.predict(X_test)

    # تقييم النموذج
    rmse = np.sqrt(mean_squared_error(y_test, predictions))
    print(f'Root Mean Squared Error: {rmse}')

    # رسم النتائج
    plt.figure(figsize=(12, 6))
    plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
    plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
    plt.title('Actual vs Predicted Prices')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

    # حساب Order Blocks
    bullish_blocks, bearish_blocks = calculate_order_blocks(data)

    # حساب Premium وDiscount
    premium, discount = calculate_premium_discount(data)
    data['Premium'] = premium
    data['Discount'] = discount

    # حساب مناطق العرض والطلب
    supply_zones, demand_zones = calculate_supply_demand(data)

    # حساب خطوط الدعم والمقاومة
    support, resistance = calculate_support_resistance(data)
    data['Support'] = support
    data['Resistance'] = resistance

    # حساب مستويات فيبوناتشي
    fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
    for key, value in fib_levels.items():
        data[key] = value

    # حساب RSI
    data = calculate_rsi(data)

    # حساب Moving Average
    data = calculate_moving_average(data)

    # حساب Momentum
    data = calculate_momentum(data)

    # تحليل حجم التداول
    data = analyze_volume(data)

    # حساب موجة الذئب
    wave_points = calculate_wolf_wave(data)

    # إنشاء ميزات إضافية
    data = create_features(data)

    # حساب مؤشر القنوات السعرية
    data = calculate_price_channel(data)

    # تحسين إدارة المخاطر
    data = advanced_risk_management(data)

    # عرض النتائج
    print("Bullish Order Blocks:", bullish_blocks)
    print("Bearish Order Blocks:", bearish_blocks)
    print("Supply Zones:", supply_zones)
    print("Demand Zones:", demand_zones)

# تنفيذ الاستراتيجية
if __name__ == "__main__":
    gold_data = get_and_clean_gold_prices(2024)
    if gold_data is not None:
        build_strategy(gold_data)
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        print("Sentiment Scores:", sentiments)


In [ ]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import yfinance as yf
from sklearn.metrics import mean_squared_error  # إضافة هذا السطر

def get_and_clean_gold_prices(year):
    """Reads gold prices from Yahoo Finance for a given year, cleans the data, and prints it."""
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")
        hist.dropna(inplace=True)
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
            elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['Close'].iloc[i] > data['High'].iloc[i-1]:
                supply_zones.append((data.index[i], data['High'].iloc[i-1]))
            elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['Low'].rolling(window=20).min()
        resistance = data['High'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        data['RSI'] = ta.rsi(data['Close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = ta.sma(data['Close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = ta.mom(data['Close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
                data['Close'].iloc[i+1] < data['Close'].iloc[i] and
                data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
                data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
                data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['Close'].pct_change()
        data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['High'].rolling(window=window).max()
        data['Channel_Lower'] = data['Low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
        data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
        data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
            elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

        # رسم الرسم البياني
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

        for منطقة in مناطق_طلبات_الشراء:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

        for منطقة in مناطق_طلبات_البيع:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

        plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"An error occurred while plotting order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    try:
        # إعداد الميزات والهدف
        data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        features = data.drop(['target'], axis=1)  # الميزات
        target = data['target']  # الهدف

        # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
        for col in ['Close', 'High', 'Low']:
            features[col] = pd.to_numeric(features[col], errors='coerce')

        # إزالة الصفوف الفارغة بعد التحويل
        features.dropna(inplace=True)
        target = target[features.index]  # الحفاظ على تطابق الفهارس

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # نموذج XGBoost
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ بالأسعار
        predictions = model_xgb.predict(X_test)

        # تقييم النموذج
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # رسم النتائج
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
        plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # حساب Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # حساب Premium وDiscount
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)

        # حساب خطوط الدعم والمقاومة
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب RSI
        data = calculate_rsi(data)

        # حساب Moving Average
        data = calculate_moving_average(data)

        # حساب Momentum
        data = calculate_momentum(data)

        # تحليل حجم التداول
        data = analyze_volume(data)

        # حساب موجة الذئب
        wave_points = calculate_wolf_wave(data)

        # إنشاء ميزات إضافية
        data = create_features(data)

        # حساب مؤشر القنوات السعرية
        data = calculate_price_channel(data)

        # تحسين إدارة المخاطر
        data = advanced_risk_management(data)

        # عرض النتائج
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# مثال على كيفية استخدام الدوال
if __name__ == "__main__":
    year = 2024
    gold_data = get_and_clean_gold_prices(year)
    if gold_data is not None:
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        build_strategy(gold_data)


In [ ]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from textblob import TextBlob
import yfinance as yf
from sklearn.metrics import mean_squared_error

def get_and_clean_gold_prices(year):
    """Reads gold prices from Yahoo Finance for a given year, cleans the data, and prints it."""
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")
        hist.dropna(inplace=True)
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        print(hist)
        return hist
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
            elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['Close'].iloc[i] > data['High'].iloc[i-1]:
                supply_zones.append((data.index[i], data['High'].iloc[i-1]))
            elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['Low'].rolling(window=20).min()
        resistance = data['High'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        data['RSI'] = ta.rsi(data['Close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = ta.sma(data['Close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = ta.mom(data['Close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
                data['Close'].iloc[i+1] < data['Close'].iloc[i] and
                data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
                data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
                data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['Close'].pct_change()
        data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['High'].rolling(window=window).max()
        data['Channel_Lower'] = data['Low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
        data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
        data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
            elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

        # رسم الرسم البياني
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

        for منطقة in مناطق_طلبات_الشراء:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

        for منطقة in مناطق_طلبات_البيع:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

        plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"An error occurred while plotting order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    try:
        # إعداد الميزات والهدف
        data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        features = data.drop(['target'], axis=1)  # الميزات
        target = data['target']  # الهدف

        # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
        for col in ['Close', 'High', 'Low']:
            features[col] = pd.to_numeric(features[col], errors='coerce')

        # إزالة الصفوف الفارغة بعد التحويل
        features.dropna(inplace=True)
        target = target[features.index]  # الحفاظ على تطابق الفهارس

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # نموذج XGBoost
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ بالأسعار
        predictions = model_xgb.predict(X_test)

        # تقييم النموذج
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # رسم النتائج
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
        plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # حساب Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # حساب Premium وDiscount
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)

        # حساب خطوط الدعم والمقاومة
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب RSI
        data = calculate_rsi(data)

        # حساب Moving Average
        data = calculate_moving_average(data)

        # حساب Momentum
        data = calculate_momentum(data)

        # تحليل حجم التداول
        data = analyze_volume(data)

        # حساب موجة الذئب
        wave_points = calculate_wolf_wave(data)

        # إنشاء ميزات إضافية
        data = create_features(data)

        # حساب القنوات السعرية
        data = calculate_price_channel(data)

        # إدارة المخاطر
        data = advanced_risk_management(data)

        # رسم مناطق الطلب والبيع
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# مثال على كيفية استخدام الدوال
if __name__ == "__main__":
    year = 2024
    gold_data = get_and_clean_gold_prices(year)
    if gold_data is not None:
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        build_strategy(gold_data)


In [ ]:
!pip install MetaTrader5
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import pandas_ta as ta

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967  # أدخل رقم حسابك
password = NU6-caQp  # أدخل كلمة المرور
server = RoyalInvestment-Server  # أدخل اسم السيرفر

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def get_and_clean_gold_prices(year):
    """Reads gold prices from Yahoo Finance for a given year, cleans the data, and returns it."""
    try:
        gold = yf.Ticker("GC=F")  # Gold futures ticker symbol
        hist = gold.history(start=f"{year}-01-01", end=f"{year}-12-31")
        hist.dropna(inplace=True)
        hist.index = pd.to_datetime(hist.index, format='%Y-%m-%d')
        return hist
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def fetch_news():
    """جمع الأخبار من موقع Investing.com"""
    try:
        url = "https://www.investing.com/news/commodities-news/gold"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        news_headers = soup.select('div.news-item-header')
        news_titles = [header.text.strip() for header in news_headers]
        return news_titles
    except Exception as e:
        print(f"An error occurred while fetching news: {e}")
        return []

def analyze_sentiment(news_titles):
    """تحليل المشاعر من العناوين باستخدام TextBlob"""
    sentiments = []
    try:
        for title in news_titles:
            analysis = TextBlob(title)
            score = analysis.sentiment.polarity
            sentiments.append(score)
    except Exception as e:
        print(f"An error occurred during sentiment analysis: {e}")
    return sentiments

def calculate_order_blocks(data):
    """حساب مناطق Order Blocks"""
    bullish_blocks = []
    bearish_blocks = []
    try:
        for i in range(1, len(data)):
            if data['Close'].iloc[i] > data['Close'].iloc[i-1] and data['Close'].iloc[i-1] < data['Close'].iloc[i-2]:
                bullish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
            elif data['Close'].iloc[i] < data['Close'].iloc[i-1] and data['Close'].iloc[i-1] > data['Close'].iloc[i-2]:
                bearish_blocks.append((data.index[i-1], data['Close'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating order blocks: {e}")
    return bullish_blocks, bearish_blocks

def calculate_premium_discount(data):
    """حساب Premium وDiscount"""
    data['Close'] = pd.to_numeric(data['Close'], errors='coerce')
    data['Low'] = pd.to_numeric(data['Low'], errors='coerce')
    data['High'] = pd.to_numeric(data['High'], errors='coerce')
    data.dropna(subset=['Close', 'Low', 'High'], inplace=True)

    premium = data['Close'] - data['Low'].rolling(window=20).min()
    discount = data['High'].rolling(window=20).max() - data['Close']
    return premium, discount

def calculate_supply_demand(data):
    """حساب مناطق العرض والطلب"""
    supply_zones = []
    demand_zones = []
    try:
        for i in range(1, len(data)):
            if data['Close'].iloc[i] > data['High'].iloc[i-1]:
                supply_zones.append((data.index[i], data['High'].iloc[i-1]))
            elif data['Close'].iloc[i] < data['Low'].iloc[i-1]:
                demand_zones.append((data.index[i], data['Low'].iloc[i-1]))
    except Exception as e:
        print(f"An error occurred while calculating supply and demand zones: {e}")
    return supply_zones, demand_zones

def calculate_support_resistance(data):
    """حساب خطوط الدعم والمقاومة"""
    try:
        support = data['Low'].rolling(window=20).min()
        resistance = data['High'].rolling(window=20).max()
        return support, resistance
    except Exception as e:
        print(f"An error occurred while calculating support and resistance: {e}")
        return None, None

def fibonacci_retracement(high, low):
    """حساب مستويات فيبوناتشي"""
    diff = high - low
    return {
        'fib_retracement_0.0': low,
        'fib_retracement_23.6': low + diff * 0.236,
        'fib_retracement_38.2': low + diff * 0.382,
        'fib_retracement_50.0': low + diff * 0.5,
        'fib_retracement_61.8': low + diff * 0.618,
        'fib_retracement_100.0': high
    }

def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        data['RSI'] = ta.rsi(data['Close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data

def calculate_moving_average(data, period=20):
    """حساب المتوسط المتحرك"""
    try:
        data['MA'] = ta.sma(data['Close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating moving average: {e}")
        return data

def calculate_momentum(data, period=10):
    """حساب الزخم"""
    try:
        data['Momentum'] = ta.mom(data['Close'], length=period)
        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data

def analyze_volume(data):
    """تحليل حجم التداول لتعزيز القمم والقيعان"""
    try:
        data['Volume_MA'] = data['Volume'].rolling(window=20).mean()
        return data
    except Exception as e:
        print(f"An error occurred while analyzing volume: {e}")
        return data

def calculate_wolf_wave(data):
    """حساب موجة الذئب"""
    wave_points = []
    try:
        for i in range(1, len(data)-4):
            if (data['Close'].iloc[i] < data['Close'].iloc[i-1] and
                data['Close'].iloc[i+1] < data['Close'].iloc[i] and
                data['Close'].iloc[i+2] > data['Close'].iloc[i+1] and
                data['Close'].iloc[i+3] < data['Close'].iloc[i+2] and
                data['Close'].iloc[i+4] > data['Close'].iloc[i+3]):
                wave_points.append(data.index[i])
    except Exception as e:
        print(f"An error occurred while calculating wolf wave: {e}")
    return wave_points

def create_features(data):
    """إنشاء ميزات إضافية مثل التغيرات في السعر"""
    try:
        data['Price_Change'] = data['Close'].pct_change()
        data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))
        data['Volatility'] = data['Log_Returns'].rolling(window=20).std()
        return data
    except Exception as e:
        print(f"An error occurred while creating features: {e}")
        return data

def calculate_price_channel(data, window=20):
    """حساب مؤشر القنوات السعرية"""
    try:
        data['Channel_Upper'] = data['High'].rolling(window=window).max()
        data['Channel_Lower'] = data['Low'].rolling(window=window).min()
        data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
        return data
    except Exception as e:
        print(f"An error occurred while calculating price channel: {e}")
        return data

def advanced_risk_management(data):
    """تحسين إدارة المخاطر باستخدام استراتيجيات التحليل الفني المتقدم"""
    try:
        data['ATR'] = ta.atr(data['High'], data['Low'], data['Close'], length=14)
        data['Stop_Loss'] = data['Close'] - (data['ATR'] * 1.5)  # وقف خسارة عند 1.5 من ATR
        data['Take_Profit'] = data['Close'] + (data['ATR'] * 3)    # جني الأرباح عند 3 من ATR
        return data
    except Exception as e:
        print(f"An error occurred while managing risk: {e}")
        return data

def احسب_وارسم_مناطق_طلبات_الشراء_والبيع(البيانات):
    """ يحسب ويرسم مناطق طلبات الشراء والبيع (Order Blocks) على الرسم البياني. """
    مناطق_طلبات_الشراء = []
    مناطق_طلبات_البيع = []

    try:
        for i in range(2, len(البيانات)):
            if البيانات['Close'].iloc[i] < البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] > البيانات['Close'].iloc[i-2]:
                مناطق_طلبات_الشراء.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))
            elif البيانات['Close'].iloc[i] > البيانات['Close'].iloc[i-1] and البيانات['Close'].iloc[i-1] < البيانات['Close'].iloc[i-2]:
                مناطق_طلبات_البيع.append((البيانات.index[i-1], البيانات['Close'].iloc[i-1]))

        # رسم الرسم البياني
        plt.figure(figsize=(12, 6))
        plt.plot(البيانات.index, البيانات['Close'], label='سعر الإغلاق', color='blue')

        for منطقة in مناطق_طلبات_الشراء:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='red', label='طلب شراء' if منطقة == مناطق_طلبات_الشراء[0] else "")

        for منطقة in مناطق_طلبات_البيع:
            index, price = منطقة
            plt.plot(index, price, marker='o', markersize=8, color='green', label='طلب بيع' if منطقة == مناطق_طلبات_البيع[0] else "")

        plt.title('مناطق طلبات الشراء والبيع على الرسم البياني')
        plt.xlabel('التاريخ')
        plt.ylabel('السعر')
        plt.legend()
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"An error occurred while plotting order blocks: {e}")

def build_strategy(data):
    """بناء الاستراتيجية الكاملة"""
    try:
        # إعداد الميزات والهدف
        data['target'] = data['Close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        features = data.drop(['target'], axis=1)  # الميزات
        target = data['target']  # الهدف

        # تحويل أعمدة 'Close', 'High', 'Low' إلى أرقام
        for col in ['Close', 'High', 'Low']:
            features[col] = pd.to_numeric(features[col], errors='coerce')

        # إزالة الصفوف الفارغة بعد التحويل
        features.dropna(inplace=True)
        target = target[features.index]  # الحفاظ على تطابق الفهارس

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # نموذج XGBoost
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ بالأسعار
        predictions = model_xgb.predict(X_test)

        # تقييم النموذج
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # رسم النتائج
        plt.figure(figsize=(12, 6))
        plt.plot(y_test.index, y_test, label='Actual Prices', color='blue')
        plt.plot(y_test.index, predictions, label='Predicted Prices', color='orange')
        plt.title('Actual vs Predicted Prices')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.legend()
        plt.show()

        # حساب Order Blocks
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # حساب Premium وDiscount
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)

        # حساب خطوط الدعم والمقاومة
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['High'].max(), data['Low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب RSI
        data = calculate_rsi(data)

        # حساب Moving Average
        data = calculate_moving_average(data)

        # حساب Momentum
        data = calculate_momentum(data)

        # تحليل حجم التداول
        data = analyze_volume(data)

        # حساب موجة الذئب
        wave_points = calculate_wolf_wave(data)

        # إنشاء ميزات إضافية
        data = create_features(data)

        # حساب القنوات السعرية
        data = calculate_price_channel(data)

        # إدارة المخاطر
        data = advanced_risk_management(data)

        # رسم مناطق الطلب والبيع
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# مثال على كيفية استخدام الدوال
if __name__ == "__main__":
    year = 2023
    gold_data = get_and_clean_gold_prices(year)
    if gold_data is not None:
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)

        # بناء الاستراتيجية
        build_strategy(gold_data)

# إغلاق الاتصال
mt5.shutdown()
